### Model of QARIB Transformer.

> Indented block



In [39]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 705 kB 5.0 MB/s 
     |████████████████████████████████| 5.9 MB 61.5 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.


## importing libaraires


In [41]:

# Add environment Packages paths to conda
import os, sys, warnings
import pandas as pd
import numpy as np
warnings.simplefilter("ignore")

# Text preprocessing packages
import nltk # Text libarary
# nltk.download('stopwords')
import string # Removing special characters {#, @, ...}

# Modelling
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.svm import SVC
# Saving Model
import pickle

# Visualization Packages
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1.3)
%matplotlib inline


from pylab import rcParams
import joblib

from transformers import AutoTokenizer, AutoModel
import torch
from torch import nn,optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics.functional import f1_score
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report

from tqdm.auto import tqdm

torch.cuda.empty_cache()


In [42]:
torch.cuda.empty_cache()

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
#import the dataset
path_input = '/content/drive/MyDrive/Colab Notebooks/cleaned_data_IRIS.csv'
df = pd.read_csv(path_input)
df.dropna(inplace=True)
df.head()

,text,label
0,اوليمبياد الجايه هكون لسه الكليه,none
1,عجز الموازنه وصل ل الناتج المحلي يعني لسه اقل ...,anger
2,كتنا نيله حظنا الهباب يضحك,sadness
3,جميعنا نريد تحقيق اهدافنا تونس تالقت حراسه المرمي,joy
4,اوليمبياد نظامها مختلف ومواعيد المونديال مكانت...,none


In [45]:
#check for null values
df.isnull().sum()

text     0
label    0
dtype: int64

In [46]:
#check for duplication
df.duplicated().sum()

0

In [47]:
#make train and test data and save it as a csv file
train, val = train_test_split(df[['label','text']], test_size=0.2)


lbl_enc = LabelEncoder()
train.loc[:,"label"] = lbl_enc.fit_transform(train["label"])
val.loc[:,"label"] = lbl_enc.transform(val["label"])

joblib.dump(lbl_enc,"label_encoder.pkl")

train.to_csv("train.csv",index=False)
val.to_csv("test.csv",index=False)

In [48]:
#print the classes and it's encoding
lbl_enc.classes_
{v: k for v, k in enumerate(lbl_enc.classes_)}

{0: 'anger',
 1: 'fear',
 2: 'joy',
 3: 'love',
 4: 'none',
 5: 'sadness',
 6: 'surprise',
 7: 'sympathy'}

In [15]:
#make a new data to test different examples in our data
new_data = df.iloc[:2]
new_data

new_text  = {'text': 'حب','label': 'love'}
new_data = new_data.append(new_text, ignore_index = True)
new_text  = {'text': 'مستغرب','label': 'surprise'}
new_data = new_data.append(new_text, ignore_index = True)
new_text  = {'text': 'حزن','label': 'sadness'}
new_data = new_data.append(new_text, ignore_index = True)
new_data.loc[:,"label"] = lbl_enc.transform(new_data["label"])
new_data.to_csv("new_data.csv",index=False)

new_data


,text,label
0,اوليمبياد الجايه هكون لسه الكليه,4
1,عجز الموازنه وصل ل الناتج المحلي يعني لسه اقل ...,0
2,حب,3
3,مستغرب,6
4,حزن,5


In [49]:
class ArabicDataset(Dataset):
    def __init__(self,data,max_len,model_type="Twitter"):
        super().__init__()
        self.labels = data["label"].values
        self.texts = data["text"].values
        self.max_len = max_len
        model = {"Twitter": "aubmindlab/bert-base-arabertv02-twitter",
                "ARBERT": "UBC-NLP/ARBERT",
                "Base": "aubmindlab/bert-base-arabertv02",
                "MARBERT": "UBC-NLP/MARBERT",
                "QARIB":"qarib/bert-base-qarib"}
        self.tokenizer = AutoTokenizer.from_pretrained(model[model_type])
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self,idx):
        text = " ".join(self.texts[idx].split())
        label = self.labels[idx]
        inputs = self.tokenizer(text,padding='max_length',
                                max_length=self.max_len,truncation=True,return_tensors="pt")
        
        #input_ids,token_type_ids,attention_mask
        return {
            "inputs":{"input_ids":inputs["input_ids"][0],
                      "token_type_ids":inputs["token_type_ids"][0],
                      "attention_mask":inputs["attention_mask"][0],
                     },
            "labels": torch.tensor(label,dtype=torch.long) 
        }

In [50]:
class ArabicDataModule(pl.LightningDataModule):
    def __init__(self,train_path,val_path,new_data_path,batch_size=12,max_len=100,model_type="Twitter"):
        super().__init__()
        self.train_path,self.val_path,self.new_data_path= train_path,val_path,new_data_path
        self.batch_size = batch_size
        self.max_len = max_len
        self.model_type = model_type
    
    def setup(self,stage=None):
        train = pd.read_csv(self.train_path)
        val = pd.read_csv(self.val_path)
        new_data = pd.read_csv(self.new_data_path)
        self.train_dataset = ArabicDataset(data=train,max_len=self.max_len,model_type=self.model_type)
        self.val_dataset = ArabicDataset(data=val,max_len=self.max_len,model_type=self.model_type)
        self.new_dataset = ArabicDataset(data=new_data,max_len=self.max_len,model_type=self.model_type)

    def train_dataloader(self):
        return DataLoader(self.train_dataset,batch_size=self.batch_size,shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.new_dataset,batch_size=self.batch_size,shuffle=False)
    
    def test_dataloader(self):
        return DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=False)

In [51]:
n_classes = 8
class ArabicBertModel(pl.LightningModule):
    def __init__(self,model_type="Twitter"):
        super().__init__()
        model = {"Twitter": ("aubmindlab/bert-base-arabertv02-twitter",768),
                "ARBERT": ("UBC-NLP/ARBERT",768),
                 "Base": ("aubmindlab/bert-base-arabertv02",768),
                "MARBERT": ("UBC-NLP/MARBERT",768),
                "QARIB":("qarib/bert-base-qarib",768)}
        self.bert_model = AutoModel.from_pretrained(model[model_type][0])
        self.fc = nn.Linear(model[model_type][1],n_classes)
    
    def forward(self,inputs):
        out = self.bert_model(**inputs)#inputs["input_ids"],inputs["token_type_ids"],inputs["attention_mask"])
        last_hidden_states = out[1]
        out = self.fc(last_hidden_states)
        return out
    
    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr=1e-6)
    
    def criterion(self,output,target):
        return nn.CrossEntropyLoss()(output,target)
    
    #TODO: adding metrics
    def training_step(self,batch,batch_idx):
        x,y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_result = f1_score(out, y, num_classes=n_classes)
        metrics = {"train_f1": f1_result, "train_loss": loss}
        self.log_dict(metrics)
        return loss
    
    def validation_step(self,batch,batch_idx):
        x, y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_result = f1_score(out, y, num_classes=n_classes)
        metrics = {"val_f1": f1_result, "val_loss": loss}
        self.log_dict(metrics)
        return metrics

# Stop

## Training on QARIB (10 epoch)

In [19]:
MODEL_TYPE = "QARIB"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./test.csv",
                new_data_path = "./new_data.csv",
                batch_size=128, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=10, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_loss")]) 
trainer.fit(model,dm)

Some weights of the model checkpoint at qarib/bert-base-qarib were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDepr

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type      | Params
-----------------------------------------
0 | bert_model | BertModel | 135 M 
1 | fc         | Linear    | 6.2 K 
-----------------------------------------
135 M     Trainable params
0         Non-trainable params
135 M     Total params
540.798   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))

  0%|          | 0/16 [00:00<?, ?it/s]

              precision    recall  f1-score   support

       anger       0.66      0.79      0.72       290
        fear       0.88      0.91      0.89       242
         joy       0.58      0.34      0.43       244
        love       0.71      0.80      0.75       249
        none       0.54      0.97      0.69       279
     sadness       0.66      0.30      0.41       238
    surprise       0.58      0.28      0.38       191
    sympathy       0.76      0.80      0.78       203

    accuracy                           0.67      1936
   macro avg       0.67      0.65      0.63      1936
weighted avg       0.67      0.67      0.64      1936



## Training on QARIB (5 epoch)




In [ ]:
MODEL_TYPE = "QARIB"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./test.csv",
                new_data_path = "./new_data.csv",
                batch_size=128, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=5, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_loss")]) 
trainer.fit(model,dm)

Some weights of the model checkpoint at qarib/bert-base-qarib were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))

  0%|          | 0/16 [00:00<?, ?it/s]

              precision    recall  f1-score   support

       anger       0.51      0.78      0.62       290
        fear       0.68      0.84      0.75       242
         joy       0.54      0.15      0.24       244
        love       0.57      0.84      0.68       249
        none       0.53      0.98      0.68       279
     sadness       0.42      0.03      0.06       238
    surprise       0.80      0.06      0.12       191
    sympathy       0.75      0.78      0.77       203

    accuracy                           0.58      1936
   macro avg       0.60      0.56      0.49      1936
weighted avg       0.59      0.58      0.50      1936



## Save The Model

In [23]:
torch.save(model, 'QARIB_arabic_dialect.pth')

## Test on new data

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load('/content/QARIB_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.val_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()


  0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
lbl_enc.classes_
{v: k for v, k in enumerate(lbl_enc.classes_)}

{0: 'anger',
 1: 'fear',
 2: 'joy',
 3: 'love',
 4: 'none',
 5: 'sadness',
 6: 'surprise',
 7: 'sympathy'}

In [28]:
new_data

,text,label
0,اوليمبياد الجايه هكون لسه الكليه,4
1,عجز الموازنه وصل ل الناتج المحلي يعني لسه اقل ...,0
2,حب,3
3,مستغرب,6
4,حزن,5


# what about if we decrease the batch size and we increase number of epochs

In [29]:
MODEL_TYPE = "QARIB"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./test.csv",
                new_data_path = "./new_data.csv",
                batch_size=50, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=30, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_loss")]) 
trainer.fit(model,dm)

Some weights of the model checkpoint at qarib/bert-base-qarib were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDepr

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.


In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))

  0%|          | 0/39 [00:00<?, ?it/s]

              precision    recall  f1-score   support

       anger       0.75      0.78      0.76       290
        fear       0.97      0.90      0.94       242
         joy       0.66      0.69      0.67       244
        love       0.82      0.81      0.81       249
        none       0.71      0.72      0.72       279
     sadness       0.59      0.59      0.59       238
    surprise       0.54      0.55      0.54       191
    sympathy       0.93      0.89      0.91       203

    accuracy                           0.74      1936
   macro avg       0.75      0.74      0.74      1936
weighted avg       0.75      0.74      0.75      1936



## What about adjusting the learning rate?

I will decrease the learning rate to 2e-06 and I will see the results of training with different number of epochs

In [31]:
n_classes = 8
class ArabicBertModel(pl.LightningModule):
    def __init__(self,model_type="Twitter"):
        super().__init__()
        model = {"Twitter": ("aubmindlab/bert-base-arabertv02-twitter",768),
                "ARBERT": ("UBC-NLP/ARBERT",768),
                 "Base": ("aubmindlab/bert-base-arabertv02",768),
                "MARBERT": ("UBC-NLP/MARBERT",768),
                 "QARIB":("qarib/bert-base-qarib",768)}
        self.bert_model = AutoModel.from_pretrained(model[model_type][0])
        self.fc = nn.Linear(model[model_type][1],n_classes)
    
    def forward(self,inputs):
        out = self.bert_model(**inputs)#inputs["input_ids"],inputs["token_type_ids"],inputs["attention_mask"])
        last_hidden_states = out[1]
        out = self.fc(last_hidden_states)
        return out
    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr=2e-06)
    
    def criterion(self,output,target):
        return nn.CrossEntropyLoss()(output,target)
    
    #TODO: adding metrics
    def training_step(self,batch,batch_idx):
        x,y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_result = f1_score(out, y, num_classes=n_classes)
        metrics = {"train_f1": f1_result, "train_loss": loss}
        self.log_dict(metrics)
        return loss
    
    def validation_step(self,batch,batch_idx):
        x, y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_result = f1_score(out, y, num_classes=n_classes)
        metrics = {"val_f1": f1_result, "val_loss": loss}
        self.log_dict(metrics)
        return metrics

Training QARIB With new lr and (10 epoch)

In [32]:
# TODO: getting different models sizes results
MODEL_TYPE = "QARIB"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./test.csv",
                new_data_path = "./new_data.csv",
                batch_size=128, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=10, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_loss")]) 
trainer.fit(model,dm)

Some weights of the model checkpoint at UBC-NLP/ARBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=-

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type      | Params
-----------------------------------------
0 | bert_model | BertModel | 162 M 
1 | fc         | Linear    | 6.2 K 
-----------------------------------------
162 M     Trainable params
0         Non-trainable params
162 M     Total params
651.390   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))



  0%|          | 0/16 [00:00<?, ?it/s]

              precision    recall  f1-score   support

       anger       0.70      0.75      0.73       290
        fear       0.97      0.89      0.93       242
         joy       0.62      0.54      0.58       244
        love       0.73      0.80      0.76       249
        none       0.65      0.81      0.72       279
     sadness       0.54      0.52      0.53       238
    surprise       0.58      0.40      0.47       191
    sympathy       0.82      0.86      0.84       203

    accuracy                           0.70      1936
   macro avg       0.70      0.70      0.69      1936
weighted avg       0.70      0.70      0.70      1936



## Training on QARIB with the new LR (20 epoch)


In [34]:
# TODO: getting different models sizes results
MODEL_TYPE = "QARIB"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./test.csv",
                new_data_path = "./new_data.csv",
                batch_size=128, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_loss")]) 
trainer.fit(model,dm)

Some weights of the model checkpoint at UBC-NLP/ARBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=-

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))



  0%|          | 0/16 [00:00<?, ?it/s]

              precision    recall  f1-score   support

       anger       0.75      0.71      0.73       290
        fear       0.96      0.90      0.93       242
         joy       0.62      0.59      0.61       244
        love       0.78      0.76      0.77       249
        none       0.65      0.81      0.72       279
     sadness       0.59      0.55      0.57       238
    surprise       0.49      0.50      0.49       191
    sympathy       0.91      0.89      0.90       203

    accuracy                           0.72      1936
   macro avg       0.72      0.71      0.71      1936
weighted avg       0.72      0.72      0.72      1936



# what about if we decrease the batch size and we increase number of epochs

In [36]:
# TODO: getting different models sizes results
MODEL_TYPE = "QARIB"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./test.csv",
                new_data_path = "./new_data.csv",
                batch_size=50, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=30, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_loss")]) 
trainer.fit(model,dm)

Some weights of the model checkpoint at UBC-NLP/ARBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=-

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.


In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))



  0%|          | 0/39 [00:00<?, ?it/s]

              precision    recall  f1-score   support

       anger       0.75      0.66      0.70       290
        fear       0.95      0.89      0.92       242
         joy       0.63      0.55      0.59       244
        love       0.74      0.80      0.77       249
        none       0.69      0.71      0.70       279
     sadness       0.52      0.62      0.57       238
    surprise       0.45      0.49      0.47       191
    sympathy       0.93      0.88      0.91       203

    accuracy                           0.70      1936
   macro avg       0.71      0.70      0.70      1936
weighted avg       0.71      0.70      0.71      1936



# The End 